<a href="https://colab.research.google.com/github/NiclasFenton-Wiegleb/schlager-lyrics-bot/blob/main/Schlage_bot_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==4.3
!pip install datasets
!pip install peft
!pip install accelerate
!pip install bitsandbytes
!pip install trl
!pip install safetensors


In [2]:
#Check CUDA is running on GPU and bf16 supported (required)

import torch
print(torch.cuda.is_available(), torch.cuda.is_bf16_supported())

True False


In [3]:
#Import dependencies

from random import randrange
import pandas as pd
from datasets import Dataset

In [4]:
df_lyrics = pd.read_csv('schlager_songs_v2.csv')

df_lyrics['lyrics'][10]

'{"status"false,"reason""Unexpected error occurred (no quota cost) Please try again later"}'

In [5]:
# Create separate column for 1st verse

verse_list = []

for ind in df_lyrics["lyrics"].index:
    lyrics = df_lyrics["lyrics"].iloc[ind]
    if lyrics == '{"status"false,"reason""Unexpected error occurred (no quota cost) Please try again later"}' :
        verse_list.append(None)
    else:
        try:
            verse = lyrics.split("\n")[0]
            n = 1
            while len(' '.join(verse).split()) <= 15:
                verse = lyrics.split("\n")[0:n]
                n += 1
            verse_list.append(''.join(verse))

        except:
            verse_list.append(None)


df_lyrics["verse_1"] = verse_list

df_lyrics["verse_1"][0]

df_lyrics.to_csv("schlager_songs_v2.csv")

In [6]:
# Prompt Example

### Instruction:

"Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben."

### Input:

"Du sagst nicht ein Wort"

### Response:

f""" Du sagst nicht ein Wort
 Und deine Hand wischt eine Träne fort
 Und dein leerer Blick
 Sinkt in dein Glas
 Du sitzt hier vor mir
 Und dein Gesicht lässt keinen Zweifel mehr
 Heut sagst du mir
 Dass ich dich verlier'
 Nie war Zeit für dich
 Ich lebte nur in meiner eignen Welt
 Ich weiß, du wirst gehen
 Ich muss dich versteh'n
 Lieb mich ein letztes mal
 Es bleibt mir keine andre Wahl
 Ich weiß, dass ich die Nacht mit dir
 An den Tag verlier'
...
 Und bleib bei mir"""

" Du sagst nicht ein Wort\n Und deine Hand wischt eine Träne fort\n Und dein leerer Blick\n Sinkt in dein Glas\n Du sitzt hier vor mir\n Und dein Gesicht lässt keinen Zweifel mehr\n Heut sagst du mir\n Dass ich dich verlier'\n Nie war Zeit für dich\n Ich lebte nur in meiner eignen Welt\n Ich weiß, du wirst gehen\n Ich muss dich versteh'n\n Lieb mich ein letztes mal\n Es bleibt mir keine andre Wahl\n Ich weiß, dass ich die Nacht mit dir\n An den Tag verlier'\n...\n Und bleib bei mir"

In [7]:
def format_instruction(sample):

    output_texts = []

    for i in range(len(sample['input'])):
      text = f"""### Anweisung:
Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben.

### Input:

{sample['input']}

### Output:

{sample['output']}"""

      output_texts.append(text)

    return output_texts


In [8]:
#Create input and output dataset from dataframe

df_dataset = df_lyrics[["verse_1", "lyrics"]]

ind_nan = []

for ind in df_dataset.index:
    if df_dataset["verse_1"].iloc[ind] == None:
        ind_nan.append(ind)
    else:
        continue

df_dataset.drop(index=ind_nan, inplace=True)

df_dataset.reset_index(drop=True, inplace=True)

df_dataset.rename(columns={"lyrics" : "output", "verse_1" : "input"}, inplace= True)

<ipython-input-8-933d3ca1faf6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset.drop(index=ind_nan, inplace=True)
<ipython-input-8-933d3ca1faf6>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dataset.rename(columns={"lyrics" : "output", "verse_1" : "input"}, inplace= True)


In [9]:
print(len(df_dataset))

1046


In [10]:
train_df = df_dataset[0:946]
test_df = df_dataset[947:1046]

In [11]:
from datasets.dataset_dict import DatasetDict

datasets_train_test = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "test": Dataset.from_pandas(test_df)
    })

datasets_train_test

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 946
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 99
    })
})

In [12]:
print(format_instruction(datasets_train_test["train"][randrange(946)])[0])

### Anweisung:
Benuzte den unten gegebenen Vers um den Text für ein Lied zu schreiben.
      
### Input:
      
 Ich bleib einfach hier heute Nacht
      
### Output:
      
 Ich bleib einfach hier heute Nacht
 Trink Bier heute Nacht
 Mit dir, wenn de Bock hast
 Bei allen anderen sind schon die Leuchten aus
 Doch nicht mal zehn Türsteher kriegen mich heut hier raus
 Mit jeder Stunde steigt hier mein Verbrauch
 Leg mich unter den Zapfhahn und trink weiter, ow
 Und wenn der Hahn nicht mehr läuft, dann nehm ich mir 'n Kasten
 Ich bin zwar schon über , doch noch lang nicht erwachsen
 Die ganze Kohle hau ich einfach raus
 Ich geh nirgendwo hin und schon gar nicht nach Haus
 Ich bleib einfach hier heute Nacht
 Trink Bier heute Nacht
 Mit dir, wenn de Bock hast
 Ich bleib einfach hier heute Nacht
 Rasier heute Nacht
 Mit dir, wenn de Bock hast
 Ich bleib einfach hier heute Nacht
 Trink Bier heute Nacht
 Mit dir, wenn de Bock hast
 Ich bleib einfach hier heute Nacht
 Rasier heute Nacht
 Mit di

*Break Start*

In [13]:
# Import dependencies

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import accelerate

#SFTTrainer supports a native integration with peft, which makes it easy to instruction tune LLMs

from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType


In [28]:
#Parameters
device = "cuda"
model_id = "malteos/bloom-6b4-clp-german" # non-gated

peft_config = PromptTuningConfig(
            task_type="CAUSAL_LM",
            prompt_tuning_init=PromptTuningInit.TEXT,
            num_virtual_tokens=8,
            prompt_tuning_init_text="Benuzte den gegebenen Vers um den Text für ein Schlager Lied zu schreiben.",
            tokenizer_name_or_path=model_id
)

dataset_name = "schlager_lyrics"
checkpoint_name = f"{dataset_name}_{model_id}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/", "_"
)
text_column = "input"
label_column = "output"
max_length = 64 #optimize
lr = 2e-4
num_epochs = 50
batch_size = 4


Preprocess dataset

In [29]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [30]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{x}" for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(datasets_train_test["train"][0:10])

In [32]:
processed_datasets = datasets_train_test.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=datasets_train_test["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/946 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/99 [00:00<?, ? examples/s]

In [ ]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["test"]


train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=batch_size,
    pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

*Break End*

In [14]:
# Hugging Face model id
model_id = "malteos/bloom-6b4-clp-german" # non-gated


# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


bloom_model = BloomModelForCausalLM.from_pretrained(
      model_id,
      load_in_8bit=True,
      # quantization_config=bnb_config,
      use_cache=False,
      torch_dtype=torch.float16,
      device_map="auto")

# model.config.pretraining_tp = 1


Loading checkpoint shards:   0%|          | 0/32 [00:00<?, ?it/s]

In [15]:
#Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

In [16]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=64,
        bias="none",
        task_type="CAUSAL_LM"
)


# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [17]:
#We need to define the hyperparameters before we can start training

args = TrainingArguments(
    output_dir="./content/drive/MyDrive/AiCore/trained_model/schlager-bot-001",
    num_train_epochs=3,
    per_device_train_batch_size= 4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    # bf16=True,
    # tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    # disable_tqdm=True  # disable tqdm since with packing values are in correct
)

In [ ]:
#We can now configure the trainer

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    # peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args,
)

Map:   0%|          | 0/1046 [00:00<?, ? examples/s]

In [ ]:
# train
trainer.train() # there will not be a progress bar since tqdm is disabled

# save model
trainer.save_model()